# 千帆function_call工具调用

## 简介

上一节[千帆function_call入门](./function_call.ipynb)展示了实现chat调用函数的功能，本节将介绍如何让chat与千帆工具进行交互，并编写更便利的调用函数。

## 准备

本文使用的模块如下：
- 千帆 Python SDK中的 chat_completion 模块，该模块提供了与千帆对话引擎的交互接口；
- 千帆 tool中的duckduckgo_search_tool，该工具提供了网络搜索功能。

首先安装千帆 Python SDK，版本号 >= 0.2.6。

In [104]:
%%capture
!pip install "qianfan>=0.2.6" -U
!pip install duckduckgo_search

初始化我们所需要的凭证

In [105]:
# 初始化LLM
import os

# qianfan sdk 鉴权
# os.environ["QIANFAN_ACCESS_KEY"]="..."
# os.environ["QIANFAN_SECRET_KEY"]="..."


自定义函数库，此处调用qianfan.common.tool中的duckduckgo搜索引擎工具进行搜索，编写today工具查询今天日期。

In [106]:
from datetime import date
from typing import List

from qianfan.common.tool.base_tool import BaseTool, ToolParameter
from qianfan.common.tool import duckduckgo_search_tool
import json


class TodayTool(BaseTool):
    name: str = "today"
    description: str = "获取今天的日期，格式为YYYY-MM-DD"
    parameters: List[ToolParameter] = []

    def run(self, parameters: dict = {}) -> dict:
        return {
            "date": date.today().strftime("%Y-%m-%d")
        }


将工具添加到函数库中并定义函数名映射。

In [107]:
search_tool = duckduckgo_search_tool.DuckDuckGoSearchTool()
today_tool = TodayTool()

funcs = [
    search_tool.to_function_call_schema(),
    TodayTool().to_function_call_schema()
]

func_name_map = {
    search_tool.name:search_tool,
    today_tool.name:today_tool
}

将对话过程封装成函数，方便多轮对话调用

In [108]:
import qianfan

def chat(chat_completion, messages, functions, function_map):
    resp = chat_completion.do(
        messages=messages,
        functions=functions
    )
    
    while resp.get("function_call"):
        func_call = resp["function_call"]
        func_name = func_call["name"]
        arguments = json.loads(func_call["arguments"])
        func_content_dict = function_map[func_name].run(arguments)
        func_content = json.dumps(func_content_dict,ensure_ascii=False)
        
        messages.append(resp, role="assistant")
        messages.append(func_content, role="function")
        
        resp = chat_completion.do(
            messages=messages,
            functions=functions
        )
        
        print("**call function {}**".format(func_name))
        print("function arguments: {}".format(arguments))
        # print("function result: {}".format(func_content))
    resp_content = resp["result"]
    print(resp_content)
    msgs.append(resp_content, role="assistant")

开始对话，首先获取今天的日期。

In [109]:
chat_comp = qianfan.ChatCompletion(model="ERNIE-4.0-8K")
query = "今天是几号？"
msgs = qianfan.QfMessages()
msgs.append(query,role="user")
chat(chat_comp, msgs, funcs, func_name_map)

**call function today**
function arguments: {}
**No function**
根据查询，今天是2024年1月15日。如果您还有其他问题或需要更多信息，请随时告诉我。


接下来进行多轮对话，用日期信息来查询新闻。

In [110]:
query = "那么根据刚才的日期昨天有什么新闻么？搜索并请列举出标题、摘要"
msgs.append(query,role="user")
chat(chat_comp, msgs, funcs, func_name_map)

**call function duckduckgo_search**
function arguments: {'search_query': '新闻 2024-01-14'}
**No function**
很抱歉，根据我的搜索，没有找到2024年1月14日的新闻标题和摘要。如果您有其他问题或需要进一步帮助，请随时告诉我。


In [111]:
query = "还有哪些新闻？"
msgs.append(query,role="user")
chat(chat_comp, msgs, funcs, func_name_map)

**No function**
除了上述新闻，还有一些其他的新闻，包括：

1. 黄晓军院士入选法国国家医学科学院外籍院士：中国工程院院士黄晓军因在骨髓移植领域的开创性工作和杰出成就，入选法国国家医学科学院外籍院士。黄晓军长期致力于白血病等恶性血液疾病的临床和科研工作，勇于突破国际移植禁区，开创了骨髓移植“北京方案”模式，解决了骨髓移植供者来源不足的世界医学难题，大幅提高骨髓移植患者的术后生存率以及生活质量。
2. NASA执行“阿耳忒弥斯1号”任务：NASA的太空发射系统和执行“阿耳忒弥斯1号”任务的“猎户座”飞船也是最近的新闻热点。该任务是美国国家航空航天局（NASA）的阿波罗计划以来的首次载人登月任务，计划于2024年将宇航员送往月球南极附近的月球轨道，并开展一系列科学实验和探索活动。
3. 京津冀地区发生多起重大事件：在京津冀地区，也发生了多起重大事件，包括北京经开区“火箭街区”飞出“世界之最”等。此外，还有一些社会、经济、文化等方面的新闻，如某公司推出新产品、某明星涉嫌违法等。

总之，新闻种类繁多，涵盖了各个领域，人们可以通过各种渠道获取自己感兴趣的新闻信息。


In [112]:
query = "总结一下，这些新闻涉及了哪些主题？"
msgs.append(query,role="user")
chat(chat_comp, msgs, funcs, func_name_map)

**call function duckduckgo_search**
function arguments: {'search_query': '新闻主题分类'}
**No function**
根据您提供的新闻，这些新闻涉及了多个主题，包括医学、航天、社会、经济、文化等。其中，黄晓军院士入选法国国家医学科学院外籍院士的新闻涉及医学领域，NASA执行“阿耳忒弥斯1号”任务的新闻涉及航天领域，京津冀地区发生多起重大事件的新闻涉及社会、经济、文化等多个领域。
